In [ ]:
import os.path as op
import socket
import mne
from mne.time_frequency import csd_array_morlet, csd_array_fourier
import numpy as np
import itertools
from scipy.special import comb
from scipy.io import loadmat
from scipy.signal import decimate
import matplotlib.pyplot as plt

# Set how many jobs csd calculation should use
if 'node' in socket.gethostname():
    N_JOBS_CSD = 16
else:
    N_JOBS_CSD = 8

In [ ]:
subID = 'sub-01'
dest_dir = op.join('/imaging', 'ma09', 'Projects', 'AVSpeechMEG', 'data',
                   'derivatives', 'megcoherence', subID)
# Load  envelopes from the first subject
envelopes_fname = op.join(dest_dir,'envelopes.mat')
temp = loadmat(envelopes_fname)
envelopes = temp['envelopes']
sfreq = 250.0
aud_env = envelopes[:,[0],:]
vis_env = envelopes[:,[1],:]

In [ ]:
# Resample the signals for fft
do_resample = True
if do_resample:
    decim_fact = 2
    sfreq = sfreq/decim_fact
    # Decimate applies an anti-aliasing filter first!
    aud_env = decimate(aud_env, decim_fact)
    vis_env = decimate(vis_env, decim_fact)

# Compute all possible pairings of sentences, leaving pairs between same
# sentences out (N choose K problem)
idx = list(itertools.combinations(range(275),2))
idx = np.array([[x,y] for x, y in idx])

# Packing envelope pairs in arrays
temp0 = aud_env[idx[:,0],:,:]
temp1 = aud_env[idx[:,1],:,:]
aud_env_allcomb = np.concatenate((temp0,temp1),1)
temp0 = vis_env[idx[:,0],:,:]
temp1 = vis_env[idx[:,1],:,:]
vis_env_allcomb = np.concatenate((temp0,temp1),1)

# Estimate the CSD matrix
# Using morlet wavelets
# freqs_mor = np.arange(0.5,20,0.5).tolist()
# csd_mor_aud = csd_array_morlet(aud_env_allcomb, sfreq,
#                                frequencies=freqs_mor,
#                                verbose=False, n_jobs=N_JOBS_CSD)
# csd_mor_vis = csd_array_morlet(vis_env_allcomb, sfreq,
#                                frequencies=freqs_mor,
#                                verbose=False, n_jobs=N_JOBS_CSD)


In [ ]:
# Using fft
freqs_mor = np.arange(0.5,20,0.5).tolist()
csd_fft_aud = csd_array_fourier(aud_env_allcomb, sfreq,
                                fmin=freqs_mor[0], fmax=freqs_mor[-1],
                                n_fft=None, #round(aud_env_allcomb.shape[-1]/5),
                                verbose=False, n_jobs=N_JOBS_CSD)
csd_fft_vis = csd_array_fourier(vis_env_allcomb, sfreq,
                                fmin=freqs_mor[0], fmax=freqs_mor[-1],
                                n_fft=None, #round(vis_env_allcomb.shape[-1]/5),
                                verbose=False, n_jobs=N_JOBS_CSD)


In [ ]:
def coherence_spctrm(csd):
    nfreq = csd.__len__()
    coherence = []
    for ifreq in range(nfreq):
        csd_data = csd.get_data(index=ifreq)
        psd = np.diag(csd_data).real
        act_coh = np.abs(csd_data)**2 / psd[np.newaxis, :] / psd[:, np.newaxis]
        coherence.append(act_coh[1,0])
    return coherence

cohspctrm_mor_aud = coherence_spctrm(csd_mor_aud)
cohspctrm_mor_vis = coherence_spctrm(csd_mor_vis)
cohspctrm_fft_aud = coherence_spctrm(csd_fft_aud)
cohspctrm_fft_vis = coherence_spctrm(csd_fft_vis)

In [ ]:
%matplotlib inline

# plot auto-coherence spectra
fig, ax = plt.subplots(1,2,figsize=[12.8, 4.8])

ax[0].plot(freqs_mor, cohspctrm_mor_aud, label='Auditory')
ax[0].plot(freqs_mor, cohspctrm_mor_vis, label='Visual')
ax[0].set_ylim((0,1))
ax[0].legend()
ax[0].set_xlabel('Frequency (Hz)')
ax[0].set_ylabel('Coherence')
ax[0].set_title('Within-modality coherence across sentences\nMorlet wavelets')

freqs_fft = csd_fft_aud.frequencies
ax[1].plot(freqs_fft, cohspctrm_fft_aud, label='Auditory')
ax[1].plot(freqs_fft, cohspctrm_fft_vis, label='Visual')
# ax[1].set_ylim((0,1))
ax[1].legend()
ax[1].set_xlabel('Frequency (Hz)')
ax[1].set_ylabel('Coherence')
ax[1].set_title('Within-modality coherence across sentences\nFourier transform')

plt.show()
